In [29]:
# libraries
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio

In [30]:
# load data
df = pd.read_csv(r"D:\career\data_analysis\Era_Soft course\Graduation Project\Dataset\retail_store_sales.csv")

## Initial Exploration
___

In [31]:
df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,10/5/2022,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


In [33]:
df.describe()

,Price Per Unit,Quantity,Total Spent
count,11966.000000,11971.000000,11971.000000
mean,23.365912,5.536380,129.652577
std,10.743519,2.857883,94.750697
min,5.000000,1.000000,5.000000
25%,14.000000,3.000000,51.000000
50%,23.000000,6.000000,108.500000
75%,33.500000,8.000000,192.000000
max,41.000000,10.000000,410.000000


In [34]:
df.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [35]:
df.dtypes

Transaction ID       object
Customer ID          object
Category             object
Item                 object
Price Per Unit      float64
Quantity            float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
Discount Applied     object
dtype: object

In [36]:
df.duplicated().sum()

0

## During initial data exploration, I found:
- ['Item', 'price per unite', 'Quantity', 'Total spend', 'Discount Applied'] Columns have Nulls
- Incorrect data types for `transaction date`.
- Zero Dublicated rows
- Around 40% of the values in `Discount` column is null, I will handle it.

## Data cleaning & Preparation
---

### Handling missing values:

In [38]:
pd.set_option('future.no_silent_downcasting', True)
# Discount column(True if discount applied)
df['Discount Applied'] = df['Discount Applied'].fillna((df['Price Per Unit'] * df['Quantity']) > df['Total Spent'])

In [39]:
df['Item'] = df['Item'].fillna(df['Item'].mode()[0])
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Price Per Unit'].mean())
df['Quantity'] = df['Quantity'].fillna(df['Quantity'].median())
df['Total Spent'] = df['Total Spent'].fillna(df['Total Spent'].mean())

In [40]:
#check
df.isnull().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [41]:
# Renaming colums with lower case letters to work faster.
df.columns = df.columns.str.lower()
# check
df.columns

Index(['transaction id', 'customer id', 'category', 'item', 'price per unit',
       'quantity', 'total spent', 'payment method', 'location',
       'transaction date', 'discount applied'],
      dtype='object')

In [42]:
# Changing data types
df['transaction date'] = pd.to_datetime(df['transaction date'])

In [43]:
# check
df.dtypes

transaction id              object
customer id                 object
category                    object
item                        object
price per unit             float64
quantity                   float64
total spent                float64
payment method              object
location                    object
transaction date    datetime64[ns]
discount applied              bool
dtype: object

### Feature Engineering

In [44]:
# Day of week
df['day_of_week'] = df['transaction date'].dt.day_name()
# check 
df.sample()

,transaction id,customer id,category,item,price per unit,quantity,total spent,payment method,location,transaction date,discount applied,day_of_week
1679,TXN_3750711,CUST_17,Food,Item_20_FOOD,33.5,9.0,301.5,Cash,In-store,2022-06-30,False,Thursday


In [46]:
# Get season column
season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
}
df['season'] = df['transaction date'].dt.month.map(season_map)

In [48]:
# check
df.sample(5)

,transaction id,customer id,category,item,price per unit,quantity,total spent,payment method,location,transaction date,discount applied,day_of_week,season
252,TXN_8186195,CUST_20,Beverages,Item_8_BEV,15.5,5.0,77.5,Digital Wallet,Online,2023-02-19,True,Sunday,Winter
1203,TXN_5299299,CUST_18,Butchers,Item_2_BUT,6.5,6.0,39.0,Credit Card,In-store,2023-07-31,False,Monday,Summer
3942,TXN_2122383,CUST_23,Patisserie,Item_14_PAT,24.5,8.0,196.0,Digital Wallet,In-store,2023-10-07,False,Saturday,Autumn
9335,TXN_1962487,CUST_10,Milk Products,Item_15_MILK,26.0,3.0,78.0,Cash,In-store,2024-01-21,False,Sunday,Winter
755,TXN_5896151,CUST_21,Electric household essentials,Item_15_EHE,26.0,6.0,156.0,Digital Wallet,Online,2024-02-25,False,Sunday,Winter


In [50]:
# Save cleaned data
df.to_csv("cleaned_transactions.csv", index=False)